In [1]:
import numpy as np
import pandas as pd
import mlflow
import tensorflow as tf
from tensorflow import keras


2022-12-29 20:52:43.263298: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
x_train = pd.read_csv('../data/split/train/x_train.csv',index_col=0)
y_train = pd.read_csv('../data/split/train/y_train.csv',index_col=0)

x_test = pd.read_csv('../data/split/test/x_test.csv',index_col=0)
y_test = pd.read_csv('../data/split/test/y_test.csv',index_col=0)

In [3]:
train = tf.data.Dataset.from_tensor_slices((x_train,y_train))
test = tf.data.Dataset.from_tensor_slices((x_test,y_test))

2022-12-29 20:52:56.152632: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
train = train.batch(32).prefetch(tf.data.AUTOTUNE)
test = test.batch(32).prefetch(tf.data.AUTOTUNE)

In [5]:
next(iter(train))

(<tf.Tensor: shape=(32, 334), dtype=float64, numpy=
 array([[ 1.        ,  0.        , -0.28296123, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        , -1.32247844, ...,  0.        ,
          0.        ,  0.        ],
        [ 1.        ,  0.        ,  1.09629694, ...,  0.04727024,
          0.89125582,  0.        ],
        ...,
        [ 1.        ,  0.        , -0.82032844, ...,  0.        ,
          0.        ,  0.        ],
        [ 1.        ,  0.        ,  0.44631664, ...,  0.        ,
          0.        ,  0.01771176],
        [ 0.        ,  0.        , -0.48465488, ...,  0.        ,
          0.99830131,  0.        ]])>,
 <tf.Tensor: shape=(32, 1), dtype=float64, numpy=
 array([[4. ],
        [3.5],
        [4.2],
        [3.8],
        [4.2],
        [3. ],
        [3.8],
        [3. ],
        [4.4],
        [4.4],
        [4. ],
        [3.6],
        [3.7],
        [4.4],
        [4. ],
        [4. ],
        [3.2],
        [2.8],


In [6]:
model = keras.Sequential([

    keras.layers.Dense(64,activation='relu',input_shape=[334]),
    keras.layers.BatchNormalization(),
    
    keras.layers.Dense(124,activation='relu',
                       kernel_initializer=keras.initializers.glorot_normal(),
                      kernel_regularizer=keras.regularizers.L1(l1=0.01)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    
    keras.layers.Dense(256,activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    
    keras.layers.Dense(1,activation='linear')]


)


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                21440     
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 124)               8060      
                                                                 
 batch_normalization_1 (Batc  (None, 124)              496       
 hNormalization)                                                 
                                                                 
 dropout (Dropout)           (None, 124)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               3

In [11]:
keras.utils.plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [9]:
import os

In [10]:
os.system('mkdir -p deepModel_output/regression')

0

In [116]:
path = 'deepModel_output/regression'

In [117]:

check = keras.callbacks.ModelCheckpoint(path +'/weights.{epoch:02d}.hdf5',
                               save_best_only=False,save_weights_only=True)

In [118]:

output_dir='model_output/'

run_name='regression_base'
output_path=output_dir + run_name

if not os.path.exists(output_path):
    os.makedirs(output_path)
    
model_checkpoint=keras.callbacks.ModelCheckpoint(output_path + '/weights.{epoch:02d}.hdf5',
                                                 save_best_only=False,
                                                  save_weights_only=True) 

In [119]:
output_path

'model_output/regression_base'

In [120]:
root_logdir = os.path.join(os.curdir,"my_logs")

In [121]:
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
run_logdir  

'./my_logs/run_2022_12_29-20_36_44'

In [122]:
tensorboard_cb =keras.callbacks.TensorBoard(root_logdir)

In [123]:
model.compile(loss=keras.losses.MSE,optimizer='adam',
              metrics=[keras.metrics.RootMeanSquaredError()],
             )

In [124]:
history = model.fit(train,epochs=10,validation_data=test,
                   callbacks=[check,model_checkpoint,tensorboard_cb])

Epoch 1/10
836/836 [==============================] - 8s 6ms/step - loss: 8.2973 - root_mean_squared_error: 1.6989 - val_loss: 3.9512 - val_root_mean_squared_error: 0.3898
Epoch 2/10
836/836 [==============================] - 5s 7ms/step - loss: 2.7611 - root_mean_squared_error: 0.7306 - val_loss: 1.0959 - val_root_mean_squared_error: 0.3368
Epoch 3/10
836/836 [==============================] - 5s 6ms/step - loss: 0.9311 - root_mean_squared_error: 0.5896 - val_loss: 0.5133 - val_root_mean_squared_error: 0.3270
Epoch 4/10
836/836 [==============================] - 6s 7ms/step - loss: 0.6182 - root_mean_squared_error: 0.5186 - val_loss: 0.4004 - val_root_mean_squared_error: 0.3280
Epoch 5/10
836/836 [==============================] - 4s 5ms/step - loss: 0.4737 - root_mean_squared_error: 0.4686 - val_loss: 0.3232 - val_root_mean_squared_error: 0.3114
Epoch 6/10
836/836 [==============================] - 4s 5ms/step - loss: 0.3798 - root_mean_squared_error: 0.4325 - val_loss: 0.2591 - val_

In [125]:
y_pred = model.predict(x_test)

258/258 [==============================] - 1s 2ms/step


In [126]:
from sklearn.metrics import r2_score

In [127]:
r2_score(y_test,y_pred)

0.6042645194150045

In [35]:
history.history

{'loss': [3.4610157012939453,
  0.6469147801399231,
  0.3945517838001251,
  0.29459047317504883,
  0.2336430549621582,
  0.19640947878360748,
  0.1675654798746109,
  0.146799236536026,
  0.13103559613227844,
  0.11805925518274307],
 'root_mean_squared_error': [1.8603805303573608,
  0.804310142993927,
  0.6281335949897766,
  0.5427619218826294,
  0.48336637020111084,
  0.4431810975074768,
  0.40934762358665466,
  0.3831438720226288,
  0.3619883954524994,
  0.34359753131866455],
 'val_loss': [0.21797418594360352,
  0.1230018362402916,
  0.10286083072423935,
  0.09413357824087143,
  0.08716659247875214,
  0.08236849308013916,
  0.07871747016906738,
  0.07853997498750687,
  0.07424172759056091,
  0.07172266393899918],
 'val_root_mean_squared_error': [0.4668770432472229,
  0.35071617364883423,
  0.32071924209594727,
  0.30681195855140686,
  0.2952398955821991,
  0.2869991064071655,
  0.28056633472442627,
  0.28024983406066895,
  0.27247336506843567,
  0.2678108811378479]}

In [88]:
#%load_ext tensorboard


In [ ]:
#%tensorboard --logdir=./my_logs --port=6006

In [161]:
class R2Metric(keras.metrics.Metric):
    
    def __init__(self):
        self.r2 = r2_score
        self.total = 0
        self.count = 0 
        super().__init__()
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        
        metric = self.r2(y_true.numpy(), y_pred.numpy())
     
        self.total += metric
        self.count += 1
        
    def result(self):
        return self.total / self.count
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config,} 

In [162]:
model = keras.Sequential([

    keras.layers.Dense(64,activation='relu',input_shape=[334]),
    keras.layers.BatchNormalization(),
    
    keras.layers.Dense(64,activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
 
    keras.layers.Dense(1,activation='linear')]


)




In [163]:
r2 = R2Metric()
model.compile(loss=keras.losses.MSE,optimizer='adam',
              metrics=[r2],
              run_eagerly=True # to run numpy functions
             )

In [172]:
history = model.fit(train,epochs=20,validation_data=test,
                 )

Epoch 1/20
836/836 [==============================] - 192s 230ms/step - loss: 3.3827 - r2_metric_11: -21.5293 - val_loss: 0.1991 - val_r2_metric_11: -16.6239
Epoch 2/20
836/836 [==============================] - 188s 225ms/step - loss: 0.7248 - r2_metric_11: -10.8899 - val_loss: 0.1319 - val_r2_metric_11: -9.5906
Epoch 3/20
836/836 [==============================] - 185s 221ms/step - loss: 0.4440 - r2_metric_11: -7.3842 - val_loss: 0.0990 - val_r2_metric_11: -6.7717
Epoch 4/20
836/836 [==============================] - 191s 229ms/step - loss: 0.3265 - r2_metric_11: -5.5783 - val_loss: 0.0910 - val_r2_metric_11: -5.2209
Epoch 5/20
836/836 [==============================] - 195s 234ms/step - loss: 0.2580 - r2_metric_11: -4.4616 - val_loss: 0.0838 - val_r2_metric_11: -4.2258
Epoch 6/20
836/836 [==============================] - 202s 242ms/step - loss: 0.2111 - r2_metric_11: -3.6933 - val_loss: 0.0808 - val_r2_metric_11: -3.5260
Epoch 7/20
836/836 [==============================] - 199s 23

In [27]:
class keras_model(keras.models.Model,keras.layers.Layer):
    
    def __init__(self,n1=32,n2=64,number_of_hidden=1):
        
        super().__init__()
        
        self.n1 = n1
        self.n2 = n2
        self.hidden = number_of_hidden
        
        self.inp = keras.layers.Dense(self.n1,activation='relu',
                                     input_shape=[334])
        self.dense = [keras.layers.Dense(self.n2,activation='relu') 
                      for _ in range(self.hidden)]
        
        self.out = keras.layers.Dense(1,activation='linear')
        
    def call(self,X):
        
        
        x = self.inp(X)
        
        for i in range(self.hidden):
            x = self.dense[i](x)
            
        x = self.out(x)
        
        return x
            
            

In [28]:
mod = keras_model()

In [29]:
for x,y in train.take(10):
    x = x
    y= y

In [30]:
x

<tf.Tensor: shape=(32, 334), dtype=float64, numpy=
array([[ 0.        ,  0.        , -1.67082946, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        ,  0.        , -0.11532296, ...,  0.        ,
         0.86989142,  0.02269441],
       [ 0.        ,  0.        , -1.01973828, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  1.        ,  0.37941883, ...,  0.        ,
         0.        ,  0.10417623],
       [ 0.        ,  0.        , -1.56277682, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        ,  0.        , -1.67082946, ...,  0.        ,
         0.        ,  0.        ]])>

In [31]:
mod(x)

<tf.Tensor: shape=(32, 1), dtype=float32, numpy=
array([[-0.00517076],
       [-0.03515961],
       [-0.03365399],
       [ 0.12868959],
       [ 0.05268732],
       [ 0.10490862],
       [ 0.09504446],
       [ 0.18889442],
       [ 0.11253962],
       [-0.0765232 ],
       [ 0.10094057],
       [ 0.12044689],
       [ 0.161417  ],
       [ 0.16433206],
       [ 0.06888251],
       [ 0.04917868],
       [ 0.05450829],
       [ 0.04466913],
       [ 0.12830158],
       [ 0.00954445],
       [ 0.02855715],
       [ 0.06640767],
       [-0.02633381],
       [ 0.09247842],
       [ 0.06112412],
       [ 0.09665776],
       [-0.02265366],
       [ 0.02275902],
       [ 0.12482721],
       [ 0.13401881],
       [ 0.20620126],
       [ 0.10298152]], dtype=float32)>

In [32]:
mod = keras_model(number_of_hidden=2)
mod(x)

<tf.Tensor: shape=(32, 1), dtype=float32, numpy=
array([[0.07708843],
       [0.16725789],
       [0.05312563],
       [0.07799217],
       [0.05855339],
       [0.05830543],
       [0.13300273],
       [0.13985828],
       [0.15085603],
       [0.11162497],
       [0.0547572 ],
       [0.05365266],
       [0.14978483],
       [0.10343893],
       [0.15183179],
       [0.05559872],
       [0.08936116],
       [0.07109427],
       [0.21506605],
       [0.13087232],
       [0.12073077],
       [0.18239449],
       [0.12463128],
       [0.1784492 ],
       [0.20427985],
       [0.15811962],
       [0.19690359],
       [0.12880063],
       [0.13705225],
       [0.11926618],
       [0.12125569],
       [0.153039  ]], dtype=float32)>

In [40]:
def make_model(n1=32,n2=64,number_of_hidden=1,lr =0.001):
    
    model = keras_model(n1=n1,n2=n2,number_of_hidden=number_of_hidden)
    
    optim = keras.optimizers.Adam(learning_rate=lr)
 
    model.compile(loss='mse',optimizer=optim,metrics=[keras.metrics.RootMeanSquaredError()])
    
    return model

In [41]:
m = make_model()
m(x)

<tf.Tensor: shape=(32, 1), dtype=float32, numpy=
array([[-0.01810424],
       [ 0.16639733],
       [ 0.13981707],
       [ 0.15100709],
       [ 0.2849092 ],
       [ 0.00554674],
       [ 0.13321805],
       [ 0.07785926],
       [-0.03820976],
       [ 0.16829328],
       [ 0.01529009],
       [ 0.10931933],
       [ 0.0322672 ],
       [-0.04213046],
       [ 0.0900993 ],
       [ 0.07622078],
       [ 0.0530047 ],
       [ 0.08104876],
       [ 0.08833349],
       [ 0.00110856],
       [ 0.05298845],
       [ 0.15813407],
       [ 0.01024166],
       [ 0.13364324],
       [ 0.07478476],
       [ 0.10118894],
       [ 0.10557467],
       [ 0.02474373],
       [ 0.08411843],
       [ 0.11399116],
       [ 0.1610299 ],
       [ 0.04858422]], dtype=float32)>

In [42]:
m.summary()

Model: "keras_model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_29 (Dense)            multiple                  10720     
                                                                 
 dense_30 (Dense)            multiple                  2112      
                                                                 
 dense_31 (Dense)            multiple                  65        
                                                                 
Total params: 12,897
Trainable params: 12,897
Non-trainable params: 0
_________________________________________________________________


In [70]:
m.fit(train)

836/836 [==============================] - 3s 2ms/step - loss: 0.4637 - root_mean_squared_error: 0.6809


In [71]:
mod = KerasRegressor(make_model,n1=32,n2=64,number_of_hidden=1,lr =0.001)

In [74]:
from sklearn.model_selection import KFold

In [78]:
# CROSS VALIDATION FOR NN

k = KFold(n_splits=5,shuffle=True)

In [79]:
list(k.split(x_train,y_train))

[(array([    0,     1,     2, ..., 26734, 26735, 26737]),
  array([    3,     6,     9, ..., 26728, 26729, 26736])),
 (array([    0,     1,     3, ..., 26734, 26735, 26736]),
  array([    2,     8,    17, ..., 26715, 26730, 26737])),
 (array([    0,     1,     2, ..., 26734, 26736, 26737]),
  array([    4,     7,    10, ..., 26727, 26731, 26735])),
 (array([    1,     2,     3, ..., 26735, 26736, 26737]),
  array([    0,    14,    15, ..., 26719, 26725, 26732])),
 (array([    0,     2,     3, ..., 26735, 26736, 26737]),
  array([    1,     5,    11, ..., 26726, 26733, 26734]))]

In [82]:

ksplit = k.split(x_train,y_train)

In [83]:
l=[]
for train_index,test_index in ksplit:
    
    X = x_train.iloc[train_index]
    y = y_train.iloc[train_index]
    
    X_val = x_train.iloc[test_index]
    y_val = y_train.iloc[test_index]
    
    m = make_model()
    m.fit(x=X,y=y,epochs=10,batch_size=32,
          validation_data=(X_val,y_val))
    
    pred = m.predict(X_val)
    r2 = r2_score(y_val,pred)
    
    l.append(r2)

Epoch 1/10
669/669 [==============================] - 3s 4ms/step - loss: 0.7274 - root_mean_squared_error: 0.8529 - val_loss: 0.0911 - val_root_mean_squared_error: 0.3019
Epoch 2/10
669/669 [==============================] - 2s 2ms/step - loss: 0.0824 - root_mean_squared_error: 0.2871 - val_loss: 0.0833 - val_root_mean_squared_error: 0.2885
Epoch 3/10
669/669 [==============================] - 2s 2ms/step - loss: 0.0762 - root_mean_squared_error: 0.2761 - val_loss: 0.0816 - val_root_mean_squared_error: 0.2857
Epoch 4/10
669/669 [==============================] - 2s 3ms/step - loss: 0.0712 - root_mean_squared_error: 0.2668 - val_loss: 0.0751 - val_root_mean_squared_error: 0.2741
Epoch 5/10
669/669 [==============================] - 2s 3ms/step - loss: 0.0674 - root_mean_squared_error: 0.2595 - val_loss: 0.0721 - val_root_mean_squared_error: 0.2686
Epoch 6/10
669/669 [==============================] - 2s 3ms/step - loss: 0.0643 - root_mean_squared_error: 0.2535 - val_loss: 0.0720 - val_

In [84]:
l

[0.675048592198858,
 0.6621908604976778,
 0.6532552461958251,
 0.6285733658112269,
 0.643225912603139]

In [85]:
def build_model(n_hidden=1, n_neurons=10, learning_rate=0.001):
    
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=[334]))
    
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
        
    model.add(keras.layers.Dense(1,activation='relu'))
    
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    
    model.compile(loss="mse", optimizer=optimizer,
                  metrics=[keras.metrics.RootMeanSquaredError()])
    return model

In [86]:
# SKLEARN WRAPPER

from scikeras.wrappers import KerasRegressor

In [87]:
model = KerasRegressor(model=build_model,
                      n_hidden=3, n_neurons=124, learning_rate=0.001)

In [88]:
model.set_params(n_hidden=2)

KerasRegressor(
	model=<function build_model at 0x7fd5c1eebe50>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	n_hidden=2
	n_neurons=124
	learning_rate=0.001
)

In [89]:
model.fit(X=x_train,y=y_train,
         epochs=10,
          batch_size=32,
              validation_data=(x_test, y_test),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/10
836/836 [==============================] - 4s 3ms/step - loss: 0.2692 - root_mean_squared_error: 0.5188 - val_loss: 0.0842 - val_root_mean_squared_error: 0.2901
Epoch 2/10
836/836 [==============================] - 3s 3ms/step - loss: 0.0752 - root_mean_squared_error: 0.2743 - val_loss: 0.0777 - val_root_mean_squared_error: 0.2787
Epoch 3/10
836/836 [==============================] - 3s 4ms/step - loss: 0.0662 - root_mean_squared_error: 0.2572 - val_loss: 0.0910 - val_root_mean_squared_error: 0.3017
Epoch 4/10
836/836 [==============================] - 3s 3ms/step - loss: 0.0590 - root_mean_squared_error: 0.2430 - val_loss: 0.0731 - val_root_mean_squared_error: 0.2704
Epoch 5/10
836/836 [==============================] - 3s 3ms/step - loss: 0.0517 - root_mean_squared_error: 0.2275 - val_loss: 0.0784 - val_root_mean_squared_error: 0.2801
Epoch 6/10
836/836 [==============================] - 3s 3ms/step - loss: 0.0463 - root_mean_squared_error: 0.2153 - val_loss: 0.0625 - val_

KerasRegressor(
	model=<function build_model at 0x7fd5c1eebe50>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	n_hidden=2
	n_neurons=124
	learning_rate=0.001
)

In [90]:
from sklearn.metrics import r2_score

pred = model.predict(x_test)
r2_score(y_test,pred)

258/258 [==============================] - 1s 2ms/step


0.7553663125733058

In [91]:
from sklearn.model_selection import RandomizedSearchCV

In [92]:
from scipy import stats

In [102]:
stats.norm(10,20).rvs()

-9.26113671816551

In [103]:
param = dict(
n_hidden=stats.randint(3,6),
n_neurons=stats.randint(64,256), 
    
learning_rate=stats.uniform(0.001,0.01)


)

In [105]:
rand = RandomizedSearchCV(model,param,cv=5,scoring='r2')

In [108]:
# rand.fit(X=x_train,y=y_train,
#          epochs=10,
#           batch_size=32,
#               validation_data=(x_test, y_test))

In [128]:
# SAVE MODEL
model.save('best_model/neural_model')

INFO:tensorflow:Assets written to: best_model/neural_model/assets


INFO:tensorflow:Assets written to: best_model/neural_model/assets


In [129]:
mod = keras.models.load_model('best_model/neural_model')

In [130]:
pred = mod.predict(x_test)
r2_score(y_test,pred)

258/258 [==============================] - 1s 2ms/step


0.6042645194150045